# Импорт необходимых библиотек

In [110]:
from pyspark.sql.window import Window
from pyspark.sql.functions import avg, stddev, abs, col

In [126]:
import pyspark.sql.functions as F
from pyspark.sql.functions import round
from pyspark.sql import SparkSession
from pyspark.sql.window import Window

# Запуск Spark сессии

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TTTT") \
    .config("spark.jars", "/opt/spark/jars/clickhouse-jdbc-0.9.1.jar,/opt/spark/jars/postgresql-42.7.7.jar") \
    .getOrCreate()

print("Spark успешно запущен!")

Spark успешно запущен!


In [3]:
spark

In [5]:
# data = [("Test", 1)]
# df = spark.createDataFrame(data, ["Name", "Value"])
# df.show()


+----+-----+
|Name|Value|
+----+-----+
|Test|    1|
+----+-----+



In [10]:
# # Узнать текущий порт Spark UI
# print("Spark UI port:", spark.sparkContext.uiWebUrl)

# # Или получить ссылку напрямую
# spark_ui_url = spark.sparkContext.uiWebUrl
# print(f"Spark UI: {spark_ui_url}")

Spark UI port: http://77346ecdd72b:4040
Spark UI: http://77346ecdd72b:4040


In [80]:
# Узнаем текущий адрес Spark UI
ui_address = spark.sparkContext.uiWebUrl
print("Spark UI доступен по адресу:", ui_address)

# Также можно получить отдельно host и port
ui_host = spark.sparkContext.conf.get("spark.driver.host", "localhost")
ui_port = spark.sparkContext.conf.get("spark.ui.port", "4040")
print(f"Spark UI host: {ui_host}, port: {ui_port}")

Spark UI доступен по адресу: http://77346ecdd72b:4040


AttributeError: 'SparkContext' object has no attribute 'config'

# Подключаемся к источникам

## csv

In [4]:
campaigns_dict = spark.read.option("header", "true").option("inferSchema", "true").csv("/data/campaigns_dict.csv")

# Показать структуру
campaigns_dict.printSchema()

# Показать данные
campaigns_dict.show(5, truncate = False)

root
 |-- campaign_id: integer (nullable = true)
 |-- campaign_name: string (nullable = true)

+-----------+------------------------------------------+
|campaign_id|campaign_name                             |
+-----------+------------------------------------------+
|1          |year_modern_kitchen_launch_20250115       |
|2          |quarter_custom_kitchens_showcase_20240210 |
|3          |month_smart_kitchen_promotion_20240305    |
|4          |year_luxury_kitchens_exhibit_20240420     |
|5          |quarter_ecofriendly_kitchen_offer_20240512|
+-----------+------------------------------------------+
only showing top 5 rows



## parquet

In [5]:
submits = spark.read.parquet("/data/submits.parquet")


In [55]:
submits.show()

+---------+--------+-----------+
|submit_id|    name|      phone|
+---------+--------+-----------+
|     2282|Jennifer|79511904041|
|     9898| Jeffrey|79824419733|
|     9005|   Linda|79074725672|
|     1507|  Teresa|79864203598|
|     3803|   Tanya|79779567654|
|     5351| Lindsay|79818056314|
|     9262|   Holly|79492883186|
|     7363|  Sharon|79323094610|
|     4705| Jessica|79035076887|
|     6148| Gabriel|79763476258|
|     4926| Michael|79007408862|
|     3823| Patrick|79549207982|
|     4951| Cameron|79335443328|
|     2733|  Robert|79752185660|
|     1938| William|79785933427|
|     2934| Bradley|79833514346|
|       22|Michelle|79980687808|
|     6845|  Alisha|79027372303|
|      243|    Eric|79527208561|
|      534|Jennifer|79951079715|
+---------+--------+-----------+
only showing top 20 rows



In [21]:
submits.printSchema()

root
 |-- submit_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- phone: long (nullable = true)



In [9]:
import pyarrow
import pyarrow.parquet as pq

In [10]:
pq.read_metadata("/data/submits.parquet")

  created_by: parquet-cpp-arrow version 19.0.0
  num_columns: 3
  num_rows: 4000
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 2371

In [35]:
deals = spark.read.parquet("/data/deals.parquet")


In [64]:
deals.show(5, truncate = False)

+-------+----------+---------------------+-----------+------------------------+-----------------------------------------------------+
|deal_id|deal_date |fio                  |phone      |email                   |address                                              |
+-------+----------+---------------------+-----------+------------------------+-----------------------------------------------------+
|1      |2024-03-04|Gregory Wu           |79746561889|paul80@example.net      |098 Yates Cliff Apt. 241, East Monica, DE 88076      |
|2      |2024-08-20|William Ross Jr.     |79074725672|xyoung@example.org      |197 Willie Groves Apt. 655, Port Angelaberg, LA 39384|
|3      |2024-10-15|Sonya Kerr           |79201244835|elewis@example.com      |144 Andrew Cape, Lake Nicholas, SC 58918             |
|4      |2024-12-31|Mrs. Angela Tucker MD|79771829751|robertparker@example.net|6056 Collins View, South Harold, OR 15650            |
|5      |2024-03-23|Eric Flores          |79729054809|barbara7

## postgres

In [43]:
pg_host = 'localhost'
pg_port = '5432'
pg_db = 'webinar'
pg_table = 'costs'
pg_user = 'postgres'
pg_password = 'postgres'

In [6]:
# Используйте IP адрес контейнера PostgreSQL
pg_host = '172.17.0.3'    # IP адрес из docker inspect
pg_port = '5432'          # Порт PostgreSQL
pg_db = 'webinar'         # База данных
pg_user = 'postgres'      # Пользователь
pg_password = 'postgres'  # Пароль

# spark = SparkSession.builder \
#     .appName("PostgreSQL Connection") \
#     .config("spark.jars", "/opt/spark/jars/postgresql-42.7.7.jar") \
#     .getOrCreate()

# Сначала создайте тестовую таблицу
# Выполните в терминале:
# docker exec -it postgres psql -U postgres -d webinar -c "CREATE TABLE costs (date DATE, campaign_id INTEGER, costs DECIMAL(10,2), clicks INTEGER, views INTEGER); INSERT INTO costs VALUES ('2024-01-01', 1, 670.52, 40, 110);"


In [7]:

costs = (
    spark.read
    .format('jdbc')
    .option('url', f'jdbc:postgresql://{pg_host}:{pg_port}/{pg_db}')
    .option('dbtable', 'costs')
    .option('user', pg_user)
    .option('password', pg_password)
    .option('driver', 'org.postgresql.Driver')
    .load()
)

costs.show(5)

+----------+-----------+------+------+-----+
|      date|campaign_id| costs|clicks|views|
+----------+-----------+------+------+-----+
|2024-01-01|          1|670.52|    40|  110|
|2024-01-01|          2| 602.5|    11|  849|
|2024-01-01|          3|654.74|    51|  566|
|2024-01-01|          4|897.24|    86|  679|
|2024-01-01|          5|758.19|    30|  585|
+----------+-----------+------+------+-----+
only showing top 5 rows



In [34]:
costs.count()

39300

In [53]:
costs.printSchema()

root
 |-- date: date (nullable = true)
 |-- campaign_id: integer (nullable = true)
 |-- costs: double (nullable = true)
 |-- clicks: integer (nullable = true)
 |-- views: integer (nullable = true)



## clickhouse

In [24]:
ch_host = '172.17.0.1'
ch_port = '8123'
ch_db = 'default'
ch_table = 'visits_clickhouse'

In [21]:

# visits = (
#     spark.read
#     .format('jdbc')
#     .option('url', f'jdbc:clickhouse://{ch_host}:{ch_port}/{ch_db}')
#     .option('dbtable', ch_table)
#     .option('driver', 'com.clickhouse.jdbc.ClickHouseDriver')
#     .load()
# )

In [25]:
import requests
import json
from pyspark.sql.types import *

# Выполним запрос к ClickHouse через HTTP API
query = "SELECT * FROM visits_clickhouse"
url = f"http://{ch_host}:{ch_port}/?query={query}&default_format=JSONCompact"

try:
    response = requests.get(url)
    if response.status_code == 200:
        # Парсим JSON ответ
        data = response.json()
        
        # Извлекаем данные и схему
        columns = [f['name'] for f in data['meta']]
        rows = data['data']
        
        # Создаем DataFrame
        visits = spark.createDataFrame(rows, columns)
        visits.show()
    else:
        print(f"Ошибка HTTP: {response.status_code}, {response.text}")
except Exception as e:
    print(f"Ошибка при выполнении запроса: {e}")

+-------+-------------------+--------------------+--------+--------+--------+--------------------+----------------+
|visitid|      visitDateTime|                 URL|duration|clientID|  source|         UTMCampaign|          params|
+-------+-------------------+--------------------+--------+--------+--------+--------------------+----------------+
| 720128|2024-01-23 18:23:48|https://our-cool-...|      80|     289|      ad|month_openconcept...|              []|
| 695905|2024-07-24 03:32:19|https://our-cool-...|      66|     765|internal|month_contemporar...|['submit', 9469]|
| 124704|2024-05-21 15:29:08|https://our-cool-...|      91|     440|internal|month_openconcept...|              []|
| 442188|2024-01-30 21:11:15|https://our-cool-...|      65|     140| organic|month_contemporar...| ['submit', 911]|
| 644057|2024-03-11 14:05:08|https://our-cool-...|      22|     357|internal|month_openconcept...|              []|
| 511300|2024-06-16 16:41:01|https://our-cool-...|      57|     918|  di

# Готовим источники

## 1. Визиты (clickhouse)

In [26]:
visits.show()

+-------+-------------------+--------------------+--------+--------+--------+--------------------+----------------+
|visitid|      visitDateTime|                 URL|duration|clientID|  source|         UTMCampaign|          params|
+-------+-------------------+--------------------+--------+--------+--------+--------------------+----------------+
| 720128|2024-01-23 18:23:48|https://our-cool-...|      80|     289|      ad|month_openconcept...|              []|
| 695905|2024-07-24 03:32:19|https://our-cool-...|      66|     765|internal|month_contemporar...|['submit', 9469]|
| 124704|2024-05-21 15:29:08|https://our-cool-...|      91|     440|internal|month_openconcept...|              []|
| 442188|2024-01-30 21:11:15|https://our-cool-...|      65|     140| organic|month_contemporar...| ['submit', 911]|
| 644057|2024-03-11 14:05:08|https://our-cool-...|      22|     357|internal|month_openconcept...|              []|
| 511300|2024-06-16 16:41:01|https://our-cool-...|      57|     918|  di

In [27]:
filtered_visits1 = (
    visits
    .withColumn('dt', F.date_format(F.col('visitDateTime'), 'yyyy-MM-dd'))
    .where(F.col('dt').between('2024-01-01', '2025-01-27'))
    .where(F.col('source').isin('ad', 'direct'))
    .where(F.col('URL').rlike('.*checkout.*|.*add.*|.*home.*|.*contact.*|.*top50.*|.*customer-service.*|.*wishlist.*|.*sale.*|.*best-sellers.*|.*view.*|.*discount.*|.*featured.*|.*new-arrivals.*|.*settings.*|.*return-policy.*|.*edit.*|.*delete.*|.*reviews.*|.*products.*|.*about.*'))
    .select(
        'dt',
        'visitid',
        'clientID',
        'URL',
        'duration',
        'source',
        'UTMCampaign',
        'params',
        F.regexp_replace(F.col('params'), r'\[|\]', '').alias('params_regex')
    )
    .withColumn('params_split', F.split('params_regex', ', '))
)

In [208]:
filtered_visits1.show()

+----------+-------+--------+--------------------+--------+------+--------------------+----------------+--------------+----------------+
|        dt|visitid|clientID|                 URL|duration|source|         UTMCampaign|          params|  params_regex|    params_split|
+----------+-------+--------+--------------------+--------+------+--------------------+----------------+--------------+----------------+
|2024-01-23| 720128|     289|https://our-cool-...|      80|    ad|month_openconcept...|              []|              |              []|
|2024-06-16| 511300|     918|https://our-cool-...|      57|direct|month_openconcept...|['submit', 9928]|'submit', 9928|['submit', 9928]|
|2024-05-25| 453192|     114|https://our-cool-...|      64|    ad|quarter_ecofriend...|              []|              |              []|
|2024-12-03| 832703|     329|https://our-cool-...|      20|    ad|year_luxury_kitch...|              []|              |              []|
|2024-03-25| 584988|     209|https://our-

In [17]:
filtered_visits1.printSchema()

root
 |-- dt: string (nullable = true)
 |-- visitid: long (nullable = true)
 |-- clientID: long (nullable = true)
 |-- URL: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- source: string (nullable = true)
 |-- UTMCampaign: string (nullable = true)
 |-- params: string (nullable = true)
 |-- params_regex: string (nullable = true)
 |-- params_split: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [28]:
filtered_visits2 = (filtered_visits1
                        .withColumn('event_type', F.regexp_replace(F.col("params_split")[0], "'", ''))
                        .withColumn('event_id', F.col('params_split')[1].cast('int'))
                   )
                                

In [19]:
filtered_visits2.printSchema()

root
 |-- dt: string (nullable = true)
 |-- visitid: long (nullable = true)
 |-- clientID: long (nullable = true)
 |-- URL: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- source: string (nullable = true)
 |-- UTMCampaign: string (nullable = true)
 |-- params: string (nullable = true)
 |-- params_regex: string (nullable = true)
 |-- params_split: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- event_type: string (nullable = true)
 |-- event_id: integer (nullable = true)



In [29]:
visits_df = (
    filtered_visits2
    .where(F.col('event_type') == 'submit')
    .select(
        'dt',
        F.col('visitid').cast('string').alias('visitid'),
        F.col('clientID').cast('string').alias('clientid'),
        'URL',
        'duration',
        'source',
        'UTMCampaign',
        'event_type',
        'event_id'
    )
    .distinct()
)

In [25]:
visits_df.show(1, truncate = False, vertical = True)

-RECORD 0----------------------------------------------------
 dt          | 2024-07-16                                    
 visitid     | 294982                                        
 clientid    | 773                                           
 URL         | https://our-cool-website.com/customer-service 
 duration    | 57                                            
 source      | direct                                        
 UTMCampaign | year_traditional_kitchens_launch_20240707     
 event_type  | submit                                        
 event_id    | 9620                                          
only showing top 1 row



In [26]:
visits_df.count()

2433

In [27]:
visits.count()

10000

## 2. Расходы (postgres)

Рассчитаю суммы расходов, кликов и просмотрам, сгруппированные по полю campaign_id

In [38]:
costs.show(1)

+----------+-----------+------+------+-----+
|      date|campaign_id| costs|clicks|views|
+----------+-----------+------+------+-----+
|2024-01-01|          1|670.52|    40|  110|
+----------+-----------+------+------+-----+
only showing top 1 row



In [30]:
costs_df = (
    costs
    .groupBy(
        F.col('date').cast('string').alias('date'),
        'campaign_id'
    )
    .agg(
        F.sum(F.col('costs')).cast('decimal(19,2)').alias('costs'),
        F.sum(F.col('clicks')).alias('clicks'),
        F.sum(F.col('views')).alias('views')
    )
)

In [238]:
costs_df.show(2)

+----------+-----------+------+------+-----+
|      date|campaign_id| costs|clicks|views|
+----------+-----------+------+------+-----+
|2024-01-05|         75|107.03|    26|  958|
|2024-01-05|         97|408.51|    59|  292|
+----------+-----------+------+------+-----+
only showing top 2 rows



In [39]:
costs_df.printSchema()

root
 |-- date: string (nullable = true)
 |-- campaign_id: integer (nullable = true)
 |-- costs: decimal(19,2) (nullable = true)
 |-- clicks: long (nullable = true)
 |-- views: long (nullable = true)



## 3. Кампании (csv)

In [31]:
campaigns_dict.show(5, truncate=False)

+-----------+------------------------------------------+
|campaign_id|campaign_name                             |
+-----------+------------------------------------------+
|1          |year_modern_kitchen_launch_20250115       |
|2          |quarter_custom_kitchens_showcase_20240210 |
|3          |month_smart_kitchen_promotion_20240305    |
|4          |year_luxury_kitchens_exhibit_20240420     |
|5          |quarter_ecofriendly_kitchen_offer_20240512|
+-----------+------------------------------------------+
only showing top 5 rows



In [46]:
campaigns_dict.printSchema()

root
 |-- campaign_id: integer (nullable = true)
 |-- campaign_name: string (nullable = true)



In [32]:
campaigns_df = (
    campaigns_dict
    .withColumn('campaign_id', F.col('campaign_id').cast('integer'))
    .withColumn(
        'campaign_duration',
        F.when(F.col('campaign_name').like('year%'), 'Год')
        .when(F.col('campaign_name').like('quarter%'), 'Квартал')
        .when(F.col('campaign_name').like('month%'), ' Месяц')
        .otherwise(None)
    )
)

In [51]:
campaigns_df.printSchema()

root
 |-- campaign_id: integer (nullable = true)
 |-- campaign_name: string (nullable = true)
 |-- campaign_duration: string (nullable = true)



## 4. Заявки (parquet)

In [56]:
submits.printSchema()

root
 |-- submit_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- phone: long (nullable = true)



In [57]:
submits.show(1)

+---------+--------+-----------+
|submit_id|    name|      phone|
+---------+--------+-----------+
|     2282|Jennifer|79511904041|
+---------+--------+-----------+
only showing top 1 row



In [33]:
submits_df = (
    submits
    .withColumn('phone', F.col('phone').cast('string'))
    .withColumn('phone_plus', F.concat(F.lit('+'), F.col('phone')))
    .withColumn('phone_md5', F.md5('phone'))
    .withColumn('phone_plus_md5', F.md5('phone_plus'))
)

In [57]:
submits_df.show(2, truncate = False, vertical = True)

-RECORD 0------------------------------------------
 submit_id      | 2282                             
 name           | Jennifer                         
 phone          | 79511904041                      
 phone_plus     | +79511904041                     
 phone_md5      | 4c7720fdf6f9eec623dc0f961f31f488 
 phone_plus_md5 | 6ce81d5347bcd3eadb2921b7c4828e3b 
-RECORD 1------------------------------------------
 submit_id      | 9898                             
 name           | Jeffrey                          
 phone          | 79824419733                      
 phone_plus     | +79824419733                     
 phone_md5      | 9d106e45036bd4176774eb94adc9aacc 
 phone_plus_md5 | 43630233dcc965f9827e394038b0321a 
only showing top 2 rows



In [61]:
submits_df.printSchema()

root
 |-- submit_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- phone_plus: string (nullable = true)
 |-- phone_md5: string (nullable = true)
 |-- phone_plus_md5: string (nullable = true)



## 5. Сделки (parquet)

In [65]:
deals.printSchema()

root
 |-- deal_id: long (nullable = true)
 |-- deal_date: string (nullable = true)
 |-- fio: string (nullable = true)
 |-- phone: long (nullable = true)
 |-- email: string (nullable = true)
 |-- address: string (nullable = true)



In [36]:
deals_df = (
    deals
    .withColumn('username', F.split(F.col('email'), '@').getItem(0))
    .withColumn('domain', F.split(F.col('email'), '@').getItem(1))
    .where(F.col('domain').isin('example.com', 'example.org', 'example.net'))
    .withColumn('phone', F.col('phone').cast('string'))
)


In [58]:
deals_df.show(1, vertical = True)

-RECORD 0-------------------------
 deal_id   | 1                    
 deal_date | 2024-03-04           
 fio       | Gregory Wu           
 phone     | 79746561889          
 email     | paul80@example.net   
 address   | 098 Yates Cliff A... 
 username  | paul80               
 domain    | example.net          
only showing top 1 row



In [69]:
deals_df.printSchema()

root
 |-- deal_id: long (nullable = true)
 |-- deal_date: string (nullable = true)
 |-- fio: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- email: string (nullable = true)
 |-- address: string (nullable = true)
 |-- username: string (nullable = true)
 |-- domain: string (nullable = true)



# Сборка витрины

Сборка витрины по концепции "One Big Table"

In [60]:
customer_detailed = (
    visits_df.alias('v')
    .join(
        submits_df.alias('s'),
        F.col('v.event_id') == F.col('s.submit_id'),
        'left'
    )
    .join(
        deals_df.alias('d'),
        (F.col('s.phone') == F.col('d.phone')) &
        (F.col('v.dt') <= F.col('d.deal_date')),
        'left'
    )
    .join(
        campaigns_df.alias('camp'),
        F.col('v.utmcampaign') == F.col('camp.campaign_name'),
        'left'
    )
    .join(
        costs_df.alias('c'),
        (F.col('camp.campaign_id') == F.col('c.campaign_id')) &
        (F.col('v.dt') == F.col('c.date')),
        'left'
    )
    .select(
        'v.dt',
        F.col('v.visitid').alias('visit_id'),
        F.col('v.clientid').alias('client_id'),
        'v.url',
        'v.duration',
        'v.source',
        'v.utmcampaign',
        'v.event_type',
        'v.event_id',
        's.submit_id',
        's.name',
        's.phone',
        's.phone_plus',
        's.phone_md5',
        's.phone_plus_md5',
        'd.deal_id',
        'd.deal_date',
        'd.fio',
        F.col('d.phone').alias('phone_deal'),
        'd.email',
        'd.address',
        'd.username',
        'd.domain',
        'camp.campaign_name',
        'camp.campaign_duration',
        'c.costs',
        'c.clicks',
        'c.views'
    )
)

In [62]:
customer_detailed.show(1, vertical = True)

-RECORD 0---------------------------------
 dt                | 2024-09-26           
 visit_id          | 792167               
 client_id         | 565                  
 url               | https://our-cool-... 
 duration          | 33                   
 source            | ad                   
 utmcampaign       | month_spacesaving... 
 event_type        | submit               
 event_id          | 950                  
 submit_id         | 950                  
 name              | Timothy              
 phone             | 79084194784          
 phone_plus        | +79084194784         
 phone_md5         | d43cb569ec9765f04... 
 phone_plus_md5    | 0e70d155bbd56e508... 
 deal_id           | NULL                 
 deal_date         | NULL                 
 fio               | NULL                 
 phone_deal        | NULL                 
 email             | NULL                 
 address           | NULL                 
 username          | NULL                 
 domain    

In [ ]:
customer_detailed.show(1, vertical = True)

In [75]:
customer_detailed.printSchema()

root
 |-- dt: string (nullable = true)
 |-- visit_id: string (nullable = true)
 |-- client_id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- source: string (nullable = true)
 |-- utmcampaign: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- event_id: integer (nullable = true)
 |-- submit_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- phone_plus: string (nullable = true)
 |-- phone_md5: string (nullable = true)
 |-- phone_plus_md5: string (nullable = true)
 |-- deal_id: long (nullable = true)
 |-- deal_date: string (nullable = true)
 |-- fio: string (nullable = true)
 |-- phone_deal: string (nullable = true)
 |-- email: string (nullable = true)
 |-- address: string (nullable = true)
 |-- username: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- campaign_name: string (nullable = true)
 |-- campaign_duration: string (nullable = true)

In [76]:
len(customer_detailed.columns)

28

т.к будет использовать эту витрину дважды, закэшируем эту витрину

In [77]:
customer_detailed.cache()

DataFrame[dt: string, visit_id: string, client_id: string, url: string, duration: bigint, source: string, utmcampaign: string, event_type: string, event_id: int, submit_id: bigint, name: string, phone: string, phone_plus: string, phone_md5: string, phone_plus_md5: string, deal_id: bigint, deal_date: string, fio: string, phone_deal: string, email: string, address: string, username: string, domain: string, campaign_name: string, campaign_duration: string, costs: decimal(19,2), clicks: bigint, views: bigint]

In [78]:
customer_detailed.count()

2639

In [63]:
campaigns_agg = (
    customer_detailed
    .groupBy('campaign_name')
    .agg(
        F.countDistinct('visit_id').alias('unique_visits'),
        F.countDistinct('client_id').alias('unique_clients'),
        F.countDistinct('submit_id').alias('unique_submits'),
        F.countDistinct('deal_id').alias('unique_deals'),
        F.sum('costs').alias('total_costs'),
        F.sum('clicks').alias('total_clicks'),
        F.sum('views').alias('total_views'),
        F.sum('duration').alias('total_duration')
    )
    .withColumn('avg_deal_cost', (F.col('total_costs') / F.col('unique_deals')).cast('decimal(19,2)')
                                   

)
)

In [65]:
campaigns_agg.filter(F.col('total_costs').isNotNull() ).show(5, truncate = False, vertical = True)

-RECORD 0---------------------------------------------------------
 campaign_name  | year_traditional_kitchens_demo_20240725         
 unique_visits  | 22                                              
 unique_clients | 21                                              
 unique_submits | 5                                               
 unique_deals   | 1                                               
 total_costs    | 11446.75                                        
 total_clicks   | 1441                                            
 total_views    | 14004                                           
 total_duration | 1100                                            
 avg_deal_cost  | 11446.75                                        
-RECORD 1---------------------------------------------------------
 campaign_name  | month_smart_kitchens_exhibit_20240625           
 unique_visits  | 20                                              
 unique_clients | 20                                          

In [68]:
dates_agg = (
    customer_detailed
    .groupBy(F.substring('dt', 1, 7).alias('month'))
    .agg(
        F.countDistinct('visit_id').alias('unique_visits'),
        F.countDistinct('client_id').alias('unique_clients'),
        F.countDistinct('submit_id').alias('unique_submits'),
        F.countDistinct('deal_id').alias('unique_deals'),
        F.sum('costs').alias('total_costs'),
        F.sum('clicks').alias('total_clicks'),
        F.sum('views').alias('total_views'),
        F.sum('duration').alias('total_duration')
    )
    .withColumn('avg_deal_cost', (F.col('total_costs')/F.col('unique_deals')).cast('decimal(19,2)'))
)


In [70]:
dates_agg.show(1, vertical = True)

-RECORD 0-------------------
 month          | 2024-09   
 unique_visits  | 191       
 unique_clients | 168       
 unique_submits | 67        
 unique_deals   | 15        
 total_costs    | 100807.61 
 total_clicks   | 10091     
 total_views    | 102230    
 total_duration | 10225     
 avg_deal_cost  | 6720.51   
only showing top 1 row



In [71]:
dates_agg.cache().count()

13

In [73]:
def save_to_postgres(df, table_name):
    (
        df.write
        .format('jdbc')
        .option('url', f'jdbc:postgresql://{pg_host}:{pg_port}/{pg_db}')
        .option('dbtable', table_name)
        .option('user', pg_user)
        .option('password', pg_password)
        .option('driver', 'org.postgresql.Driver')
        .mode("overwrite")
        .save()
    )

In [75]:
pg_db

'webinar'

In [ ]:
Сохраним витрину данных в Postresql

In [74]:
save_to_postgres(customer_detailed, 'customer_detailed')

In [76]:
save_to_postgres(campaigns_agg, 'campaigns_agg')

In [77]:
save_to_postgres(dates_agg, 'dates_agg')

# Расчет метрик и анализ полученных данных

## 1. Кампании без выручки

In [80]:
(
campaigns_agg
.where('unique_deals = 0')
.select('campaign_name', 'total_costs', 'unique_deals')
.sort('total_costs')
.show(truncate=False)
)         

+-----------------------------------------------+-----------+------------+
|campaign_name                                  |total_costs|unique_deals|
+-----------------------------------------------+-----------+------------+
|month_smart_kitchens_launch_20240330           |7417.29    |0           |
|quarter_custom_kitchens_showcase_20240210      |8397.63    |0           |
|month_contemporary_kitchens_event_20241208     |9077.02    |0           |
|month_contemporary_kitchens_event_20240920     |10256.14   |0           |
|month_ecofriendly_kitchens_experience_20241219 |10682.85   |0           |
|quarter_ecofriendly_kitchen_experience_20241119|10999.04   |0           |
|month_smart_kitchen_promotion_20240305         |11430.88   |0           |
|year_traditional_kitchens_showcase_20241018    |12105.87   |0           |
|year_modern_kitchens_demo_20240419             |12427.38   |0           |
|month_openconcept_kitchens_initiative_20240328 |13187.82   |0           |
|quarter_luxury_kitchens_

Найдем сколько всего таких кампаний и сколько денег было потрачено на них

In [82]:
(
    campaigns_agg
    .where('unique_deals = 0')
    .select(F.countDistinct('campaign_name'), F.sum('total_costs'))
    .show()
)

+-----------------------------+----------------+
|count(DISTINCT campaign_name)|sum(total_costs)|
+-----------------------------+----------------+
|                           13|       149956.85|
+-----------------------------+----------------+



## 2. Средняя цена сделки

Посмотрим распределениее кампаний по средней цене сделки. Сортировка по убыванию

In [85]:
(
    campaigns_agg
    .where('unique_deals > 0')
    .select('campaign_name', 'total_costs', 'unique_deals', 'avg_deal_cost')
    .sort('avg_deal_cost', ascending = False)
    .show(truncate = False)
)

+-----------------------------------------------+-----------+------------+-------------+
|campaign_name                                  |total_costs|unique_deals|avg_deal_cost|
+-----------------------------------------------+-----------+------------+-------------+
|quarter_spacesaving_kitchen_innovation_20240825|16861.94   |1           |16861.94     |
|month_contemporary_kitchen_showcase_20240630   |15880.89   |1           |15880.89     |
|quarter_custom_kitchen_show_20240213           |15874.75   |1           |15874.75     |
|year_luxury_kitchens_show_20241001             |14752.07   |1           |14752.07     |
|quarter_spacesaving_kitchen_showcase_20240228  |14382.19   |1           |14382.19     |
|month_custom_kitchens_show_20241205            |13947.35   |1           |13947.35     |
|year_traditional_kitchens_launch_20240707      |13857.96   |1           |13857.96     |
|year_traditional_kitchens_experience_20240715  |12925.29   |1           |12925.29     |
|quarter_spacesaving_

## 3.Убыточные кампании, при допущении что каждая сделка стоит 5000

In [88]:
(
    campaigns_agg
    .select('campaign_name', 'total_costs', 'unique_deals', 'avg_deal_cost')
    .withColumn('revenue', F.col('unique_deals')*F.lit(5000))
    .withColumn('profit', F.col('revenue') - F.col('total_costs'))
    .sort('profit')
    .show(truncate=False)
)

+-----------------------------------------------+-----------+------------+-------------+-------+---------+
|campaign_name                                  |total_costs|unique_deals|avg_deal_cost|revenue|profit   |
+-----------------------------------------------+-----------+------------+-------------+-------+---------+
|quarter_custom_kitchens_experience_20240527    |15732.40   |0           |NULL         |0      |-15732.40|
|month_openconcept_kitchens_experience_20240910 |14837.06   |0           |NULL         |0      |-14837.06|
|quarter_luxury_kitchens_innovation_20241116    |13405.47   |0           |NULL         |0      |-13405.47|
|month_openconcept_kitchens_initiative_20240328 |13187.82   |0           |NULL         |0      |-13187.82|
|year_modern_kitchens_demo_20240419             |12427.38   |0           |NULL         |0      |-12427.38|
|year_traditional_kitchens_showcase_20241018    |12105.87   |0           |NULL         |0      |-12105.87|
|quarter_spacesaving_kitchen_innovati

## 4. Топ прибыльных кампаний, при допущении что каждая сделака стоит 5000

In [89]:
(
    campaigns_agg
    .select('campaign_name', 'total_costs', 'unique_deals', 'avg_deal_cost')
    .withColumn('revenue', F.col('unique_deals')*F.lit(5000))
    .withColumn('profit', F.col('revenue') - F.col('total_costs'))
    .sort('profit', ascending = False)
    .show(truncate=False)
)

+------------------------------------------------+-----------+------------+-------------+-------+--------+
|campaign_name                                   |total_costs|unique_deals|avg_deal_cost|revenue|profit  |
+------------------------------------------------+-----------+------------+-------------+-------+--------+
|quarter_custom_kitchens_innovation_20240817     |15356.76   |10          |1535.68      |50000  |34643.24|
|quarter_custom_kitchens_experience_20240222     |16131.74   |9           |1792.42      |45000  |28868.26|
|year_modern_kitchens_showcase_20241014          |11705.53   |8           |1463.19      |40000  |28294.47|
|month_contemporary_kitchens_promotion_20241215  |14908.49   |8           |1863.56      |40000  |25091.51|
|quarter_custom_kitchen_initiative_20241102      |16944.70   |8           |2118.09      |40000  |23055.30|
|year_traditional_kitchens_fair_20240415         |19396.53   |8           |2424.57      |40000  |20603.47|
|month_openconcept_kitchens_promotion

## 5. Рассмотрим метрики в разбивке по месяцам

In [101]:
(
    dates_agg
    .withColumn('revenue', F.col('unique_deals') * F.lit(5000))
    .withColumn('profit', F.col('revenue') - F.col('total_costs'))
    #.drop('total_duration', 'total_views')
    .sort('month')
    .show()
)

+-------+-------------+--------------+--------------+------------+-----------+------------+-----------+--------------+-------------+-------+---------+
|  month|unique_visits|unique_clients|unique_submits|unique_deals|total_costs|total_clicks|total_views|total_duration|avg_deal_cost|revenue|   profit|
+-------+-------------+--------------+--------------+------------+-----------+------------+-----------+--------------+-------------+-------+---------+
|2024-01|          204|           185|            68|          36|  114500.26|       11503|     121211|         12066|      3180.56| 180000| 65499.74|
|2024-02|          184|           164|            55|          28|   98550.71|        8960|     101263|         10213|      3519.67| 140000| 41449.29|
|2024-03|          182|           164|            57|          34|   99016.38|       10123|     103950|         10311|      2912.25| 170000| 70983.62|
|2024-04|          187|           166|            53|          21|   99450.75|        9704|   

## 6. Суммарные затраты на рекламу за год

In [92]:
dates_agg.select(F.sum('total_costs')).show()

+----------------+
|sum(total_costs)|
+----------------+
|      1319918.78|
+----------------+



##  7.Обнаружение аномалий с помощью Z-score

Z-score (стандартизированная оценка, стандартизированное значение) — статистическая мера, показывающая, на сколько стандартных отклонений наблюдение отличается от среднего значения в выборке. 

In [112]:
def detect_anomalies(df, metric_col, threshold=2.0):
    window = Window.partitionBy().rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
    df_with_stats = df.withColumn("mean", avg(metric_col).over(window)) \
                     .withColumn("stddev", stddev(metric_col).over(window))
    
    df_with_anomalies = df_with_stats.withColumn(
        "z_score", 
        abs((col(metric_col) - col("mean")) / col("stddev"))
    ).withColumn(
        "is_anomaly", 
        col("z_score") > threshold
    )
    
    return df_with_anomalies

In [117]:
campaigns_agg_with_anomalies = detect_anomalies(campaigns_agg, "total_costs", 2.5)
campaigns_agg_with_anomalies.filter('is_anomaly = true').show(truncate = False, vertical = True)

-RECORD 0-----------------------------------------------------
 campaign_name  | month_openconcept_kitchen_exhibit_20240327  
 unique_visits  | 33                                          
 unique_clients | 32                                          
 unique_submits | 12                                          
 unique_deals   | 5                                           
 total_costs    | 22668.32                                    
 total_clicks   | 2048                                        
 total_views    | 19423                                       
 total_duration | 1869                                        
 avg_deal_cost  | 4533.66                                     
 mean           | 13332.512929                                
 stddev         | 3148.7181620832885                          
 z_score        | 2.964954813492467                           
 is_anomaly     | true                                        
-RECORD 1----------------------------------------------

Вывод: Есть две кампании с аномально высокими затратами на их проведение. вероятно, эти выбросы связаны с проведением масштабных мероприятий/выставок.

In [121]:
dates_agg_with_anomalies = detect_anomalies(dates_agg, "total_costs", 2.5)
dates_agg_with_anomalies.filter('is_anomaly = true').show(vertical = True)

(0 rows)



Вывод: Нет месяцев с экстремально высокими или низкими затратами. Это хороший знак - бюджет распределен равномерно в течение года

## 8. Расчет воронки с конверсиями

In [131]:
funnel_analysis = (
    campaigns_agg
    .withColumn("visits_to_clients", 
                round(F.col("unique_clients") / F.col("unique_visits") * 100, 2))
    .withColumn("clients_to_submits", 
                round(F.col("unique_submits") / F.col("unique_clients") * 100, 2))
    .withColumn("submits_to_deals", 
                round(F.col("unique_deals") / F.col("unique_submits") * 100, 2))
    .withColumn("overall_conversion", 
                round(F.col("unique_deals") / F.col("unique_visits") * 100, 2))
    .select(
        "campaign_name",
        "unique_visits",
        "unique_clients", 
        "unique_submits",
        "unique_deals",
        "visits_to_clients",
        "clients_to_submits", 
        "submits_to_deals",
        "overall_conversion",
        "total_costs"
    )
)


Топ-10 кампаний по общей конверсии

In [130]:
funnel_analysis.orderBy(F.desc("overall_conversion")).show(10, truncate=False)

+----------------------------------------------+-------------+--------------+--------------+------------+-----------------+------------------+----------------+------------------+-----------+
|campaign_name                                 |unique_visits|unique_clients|unique_submits|unique_deals|visits_to_clients|clients_to_submits|submits_to_deals|overall_conversion|total_costs|
+----------------------------------------------+-------------+--------------+--------------+------------+-----------------+------------------+----------------+------------------+-----------+
|year_modern_kitchen_experience_20240702       |13           |13            |4             |5           |100.0            |30.77             |125.0           |38.46             |8528.47    |
|year_modern_kitchens_showcase_20241014        |23           |22            |9             |8           |95.65            |40.91             |88.89           |34.78             |11705.53   |
|month_contemporary_kitchens_promotion_202412

In [125]:
save_to_postgres(funnel_analysis, 'funnel_analysis')

Поиск узких мест в воронке

In [128]:
bottleneck_analysis = (
    funnel_analysis
    .withColumn("bottleneck_stage", 
                F.when(F.col("submits_to_deals") < 10, "Низкая конверсия из заявок в сделки")
                .when(F.col("clients_to_submits") < 10, "Низкая конверсия из клиентов в заявки")
                .when(F.col("visits_to_clients") < 10, "Низкая конверсия из визитов в клиентов")
                .otherwise("Нет явных узких мест"))
    .select("campaign_name", "bottleneck_stage", "overall_conversion")
    .orderBy("overall_conversion")
)

print("Анализ узких мест в воронке:")
bottleneck_analysis.show(truncate=False)

Анализ узких мест в воронке:
+-----------------------------------------------+-----------------------------------+------------------+
|campaign_name                                  |bottleneck_stage                   |overall_conversion|
+-----------------------------------------------+-----------------------------------+------------------+
|quarter_custom_kitchens_showcase_20240210      |Низкая конверсия из заявок в сделки|0.0               |
|year_modern_kitchens_demo_20240419             |Низкая конверсия из заявок в сделки|0.0               |
|month_ecofriendly_kitchens_experience_20241219 |Низкая конверсия из заявок в сделки|0.0               |
|month_contemporary_kitchens_event_20241208     |Низкая конверсия из заявок в сделки|0.0               |
|month_openconcept_kitchens_experience_20240910 |Низкая конверсия из заявок в сделки|0.0               |
|month_smart_kitchens_launch_20240330           |Низкая конверсия из заявок в сделки|0.0               |
|year_traditional_kitchens

# Завершение Spark session

In [93]:

customer_detailed.unpersist()
campaigns_agg.unpersist()
dates_agg.unpersist()

DataFrame[month: string, unique_visits: bigint, unique_clients: bigint, unique_submits: bigint, unique_deals: bigint, total_costs: decimal(29,2), total_clicks: bigint, total_views: bigint, total_duration: bigint, avg_deal_cost: decimal(19,2)]

In [ ]:
spark.stop()